In [ ]:
!pip install tiktoken torch

In [17]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rakibulhasanshaon69/the-verdict-txt")

print("Path to dataset files:", path)

100%|██████████| 8.86k/8.86k [00:00<00:00, 3.15MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/rakibulhasanshaon69/the-verdict-txt/versions/1


###Creating input target pairs

In [26]:
with open("/root/.cache/kagglehub/datasets/rakibulhasanshaon69/the-verdict-txt/versions/1/the-verdict.txt","r",encoding="utf-8") as f:
  raw_text = f.read()
  f.close()

In [28]:
import tiktoken
tokenizer=tiktoken.get_encoding("gpt2")

encodings = tokenizer.encode(raw_text)


In [33]:
sample_enc = encodings[50:]

In [36]:
context_len = 4

x = sample_enc[:context_len]
y = sample_enc[1:context_len+1]

In [37]:
x,y

([290, 4920, 2241, 287], [4920, 2241, 287, 257])

In [40]:
for i in range(context_len):
  input = x[:i+1]
  output = y[i]
  print(str(input)+"---->"+str(output))

[290]---->4920
[290, 4920]---->2241
[290, 4920, 2241]---->287
[290, 4920, 2241, 287]---->257


###left of the arrow refers to the token input to the llm , and right of the id is the token llms needs to predict

In [50]:
for i in range(context_len):
  input = x[:i+1]
  output = y[i:i+1]
  print(tokenizer.decode(input)+"---->"+tokenizer.decode(output))

 and----> established
 and established----> himself
 and established himself----> in
 and established himself in----> a


In [77]:
from torch.utils.data import Dataset, DataLoader
import torch
class GPTDatasetV1(Dataset):
  def __init__(self,text,tokenizer,context_len,stride):
    self.input_ids = []
    self.target_ids = []

    token_ids = tokenizer.encode(text,allowed_special={"<|endoftext|>"})

    for i in range(0,len(token_ids)-context_len,stride):
      in_chunk = token_ids[i:i+context_len]
      out_chunk = token_ids[i+1:i+context_len+1]
      self.input_ids.append(torch.tensor(in_chunk))
      self.target_ids.append(torch.tensor(out_chunk))
  def __len__(self):
      return len(self.input_ids)
  def __getitem__(self,idx):
      return self.input_ids[idx],self.target_ids[idx]

In [78]:
dataset = GPTDatasetV1(raw_text,tokenizer,4,1)

dataloader = DataLoader(dataset,batch_size=1,shuffle=False,drop_last=True,num_workers=0)

In [79]:
data_iter = iter(dataloader)

first = next(data_iter)
first

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]

256